In [ ]:
pip install optuna transformers torch tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [ ]:
pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 20.2 MB/s eta 0:00:00


In [ ]:
pip install tensorflow

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

# Load data
df = pd.read_excel("/content/Final_student_tickting.xlsx")
X = df['Reports'].values
y = df['Genre'].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Label Encoding
le = LabelEncoder()
y_train_encoder = le.fit_transform(y_train)
y_val_encoder = le.transform(y_val)

# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize data
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True, max_length=128)

# Create PyTorch datasets
train_dataset = TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(y_train_encoder)
)
val_dataset = TensorDataset(
    torch.tensor(val_encodings['input_ids']),
    torch.tensor(val_encodings['attention_mask']),
    torch.tensor(y_val_encoder)
)

# Best hyperparameters (typical for BERT fine-tuning)
best_params = {
    "lr": 2e-5,           # Learning rate
    "batch_size": 16,     # Batch size
    "epochs": 3,          # Number of epochs
    "weight_decay": 0.01  # Weight decay
}

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(le.classes_))
model.to(device)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=best_params["batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_params["batch_size"])

# Optimizer
optimizer = AdamW(model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"])

# Training loop
best_val_accuracy = 0.0
for epoch in range(best_params["epochs"]):
    model.train()
    train_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{best_params['epochs']}")

    for batch in progress_bar:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss += loss.item()
        progress_bar.set_postfix({"Training Loss": f"{loss.item():.3f}"})

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_accuracy = correct / total
    print(f"\nEpoch {epoch+1}: Validation Accuracy: {val_accuracy:.4f}")

    # Save best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), "best_model.pt")
        print(f"New best model saved with accuracy: {val_accuracy:.4f}")

print(f"\nTraining complete. Best validation accuracy: {best_val_accuracy:.4f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...


Epoch 1: Validation Accuracy: 0.9980
New best model saved with accuracy: 0.9980


Epoch 2/3: 100%|██████████| 500/500 [2:13:57<00:00, 16.08s/it, Training Loss=0.015]



Epoch 2: Validation Accuracy: 0.9960


Epoch 3/3: 100%|██████████| 500/500 [2:13:29<00:00, 16.02s/it, Training Loss=0.005]



Epoch 3: Validation Accuracy: 0.9980

Training complete. Best validation accuracy: 0.9980


In [ ]:
from sklearn.metrics import classification_report

# Reload the best model
model.load_state_dict(torch.load("best_model.pt"))
model.eval()

# Collect predictions and true labels
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Decode label indices to class names
class_names = le.classes_
report = classification_report(all_labels, all_preds, target_names=class_names)

print("\n📊 Classification Report:\n")
print(report)



📊 Classification Report:

                                   precision    recall  f1-score   support

   Academic Support and Resources       1.00      1.00      1.00       215
        Activities and Travelling       1.00      1.00      1.00       176
             Athletics and sports       1.00      0.99      0.99       171
             Career opportunities       0.99      1.00      1.00       185
                Financial Support       0.99      1.00      1.00       172
                Food and Cantines       0.99      1.00      1.00       191
    Health and Well-being Support       1.00      1.00      1.00       180
       Housing and Transportation       1.00      1.00      1.00       157
International student experiences       1.00      1.00      1.00       212
                  Online learning       1.00      1.00      1.00       172
                  Student Affairs       1.00      1.00      1.00       169

                         accuracy                           1.00      2

In [ ]:
# # Get best hyperparameters
# best_lr = study.best_params["lr"]
# best_batch_size = study.best_params["batch_size"]
# best_epochs = study.best_params["epochs"]
# best_weight_decay = study.best_params["weight_decay"]  # Make sure this matches your parameter name

# # Reinitialize model with best config
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(le.classes_))
# model.to(device)  # Move model to device before optimizer creation

# optimizer = AdamW(model.parameters(), lr=best_lr, weight_decay=best_weight_decay)
# train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=best_batch_size)

# # Add progress tracking and validation
# best_val_accuracy = 0.0
# for epoch in range(best_epochs):
#     model.train()
#     train_loss = 0.0
#     progress_bar = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{best_epochs}")

#     for batch in progress_bar:
#         input_ids, attention_mask, labels = [b.to(device) for b in batch]

#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()

#         train_loss += loss.item()
#         progress_bar.set_postfix({"Training Loss": f"{loss.item():.3f}"})

#     # Validation after each epoch
#     model.eval()
#     correct = 0
#     total = 0
#     val_loss = 0.0

#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids, attention_mask, labels = [b.to(device) for b in batch]
#             outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

#             val_loss += outputs.loss.item()
#             preds = torch.argmax(outputs.logits, dim=1)
#             correct += (preds == labels).sum().item()
#             total += labels.size(0)

#     val_accuracy = correct / total
#     avg_val_loss = val_loss / len(val_loader)
#     print(f"\nEpoch {epoch+1}: Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

#     # Save best model
#     if val_accuracy > best_val_accuracy:
#         best_val_accuracy = val_accuracy
#         torch.save(model.state_dict(), "best_model.pt")
#         print(f"New best model saved with accuracy: {best_val_accuracy:.4f}")

# print(f"\nTraining complete. Best validation accuracy: {best_val_accuracy:.4f}")